In [1]:
using ForwardDiff, Plots, Statistics, LinearAlgebra, OnlineStats, Distributions, Random, CUDA, CSV, DataFrames

using Sofia, Amlet




##################################################################################

function genFibbomod(N::Int)
    if N == 0
        return zeros(0)
    elseif N == 1
        return [0.0]
    else
        rt = [0.0, 1]
        for k in 3:N
            push!(rt, (rt[k - 2] + rt[k - 1]) % 100)
        end
        return rt
    end
end


┌ Info: Precompiling Amlet [c2ffc8e4-82bc-438e-80c0-8adc2aad9fd7]
└ @ Base loading.jl:1278
┌ Warning: Package Amlet does not have OnlineStats in its dependencies:
│ - If you have Amlet checked out for development and have
│   added OnlineStats as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Amlet
└ Loading OnlineStats into Amlet from project dependency, future warnings for Amlet are suppressed.


genFibbomod (generic function with 1 method)

In [2]:
push!(LOAD_PATH , "C:/Users/jerem/OneDrive/Desktop/GitHub/Code/FinalCode/")

using Geraldine

┌ Info: Precompiling Geraldine [c0fffcf4-b2fb-46e8-8a34-03ca56aefa66]
└ @ Base loading.jl:1278
┌ Warning: Package Geraldine does not have Distributions in its dependencies:
│ - If you have Geraldine checked out for development and have
│   added Distributions as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Geraldine
└ Loading Distributions into Geraldine from project dependency, future warnings for Geraldine are suppressed.


In [20]:
# fonctions sauvegarde 
include("saveConfig.jl")

println (generic function with 11 methods)

In [21]:
testConfigBHHH = "testConfigurationLogit_BHHH.json"
io_BHHH = open(testConfigBHHH, "w")

IOStream(<file testConfigurationLogit_BHHH.json>)

In [22]:
include("Logit\\data_10.jl");
# include("Logit\\data_100.jl");
# include("Logit\\data_1000.jl");

In [23]:
# point depart
x0 = zeros(Dim)

# for accumulator
xstar = genFibbomod(length(x0));

In [24]:
write(io_BHHH, "############################## Logit size $(Dim)  ##################################\n \n")
write(io_BHHH, " -> Size population = $(nInd)\n")
write(io_BHHH, " -> Number alternatives = $(nalt)\n")
write(io_BHHH, " -> x0 = $(x0) \n")
write(io_BHHH, " -> x optimum = $(xstar)\n")

70

In [25]:
# Fonctions de lancement 

function lancementTest(test::SecondOrderTest, allTests::AllTests)
      
    _, acc =  test.algo(allTests.model, allTests.x0, test.sam ; b = test.b, 
                        verbose = test.verbose, accumulator = test.accumulator)

    data = (getData(acc))

    if (typeof(test.sam) <: Union{Geraldine.AbstractRandomDynamicTrueVar, Geraldine.AbstractTrueVarSAA})
        append!(allTests.all_dataVariance, [data])
    else
        append!(allTests.all_dataNormal, [data])
    end
     
end

function lancementTest(test::FirstOrderTest, allTests::AllTests)
  
    _, acc = test.algo(allTests.model, test.state ; 
                        verbose = test.verbose, accumulator = test.accumulator, sp = test.sp)


    data = (getData(acc))

    append!(allTests.all_dataNormal, [data])
end

function lancementAllTests(tests::Array{Test, 1}, allTests::AllTests)
    for test in tests
       lancementTest(test, allTests) 
    end
end


lancementAllTests (generic function with 1 method)

In [57]:
 alltests = AllTests(mo, x0)


AllTests{Float64}(Amlet.LogitModel{NotUpdatable,Amlet.LinedObs}(Amlet.LogitUtility{Amlet.Linear}(Amlet.LinearUtilityForLogitModelWithCodeWellEncapsulated.u, Amlet.LinearUtilityForLogitModelWithCodeWellEncapsulated.grad, Amlet.LinearUtilityForLogitModelWithCodeWellEncapsulated.H), Amlet.LinedObs([-1.2900985484213743 -0.5567542019556669 … 0.5823251418174726 0.6342843635388268; -0.08653756583613678 -1.1566316399922312 … 1.175811629030198 0.10466501444424114; … ; 0.059685877054742466 0.7057364151832236 … 0.16756776569582307 -0.9917252722664074; -1.1379594312708283 -1.101261699617055 … 0.315429201629555 -0.14394544139421692], 5), #undef), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0, Test[], Any[], Any[], Any[], Any[], Any[], Test[], Any[], Any[], Any[], Any[], Any[])

In [58]:

Accum_2nd_tv(xstar::Vector = [0.0, 1, 1, 2, 3, 5, 8, 13, 21, 34]) = Accumulator(Value(), Iter(),
                FieldAccumulator{Float64}(:fcand), Delta(), Times(), SamplingSizeAccumulator(), DistTo(xstar),
                FieldAccumulator{Float64}(:mu), FieldAccumulator{Float64}(:sigma2), FieldAccumulator{Float64}(:sHs) ,
                FieldAccumulator{Float64}(:ρ), FieldAccumulator{Float64}(:iterCG), IsAcceptedAccumulator(), Param())

Accum_2nd_sHs(xstar::Vector = [0.0, 1, 1, 2, 3, 5, 8, 13, 21, 34]) = Accumulator(Value(), Iter(),
                FieldAccumulator{Float64}(:fcand), Delta(), Times(), SamplingSizeAccumulator(), DistTo(xstar),
                FieldAccumulator{Float64}(:mu),  FieldAccumulator{Float64}(:sHs) ,
                FieldAccumulator{Float64}(:ρ), IsAcceptedAccumulator(), Param())


Accum_2nd_sHs (generic function with 2 methods)

In [59]:
#####################################################################################
# Optimisation
verbose = false

IterMax = 10 # Nombre iterations maximales
TMax = 7200.0 # Temps max en secondes
epsilonOptimisation = 10^(-6) # Precision norme gradient

# Sampling init
N0 = 100
NMin = 50
increment = 2;

# subSampling
coeff_bhhh = 0.25
maxBhhh = 10000
subSampling = ConstantCoeffSubSampling(maxBhhh, coeff_bhhh)

#btrCoeffs = Geraldine.BTRDefaults() # avec 4*‖s‖
btrCoeffs = Geraldine.BTRDefaultsCoeff() # avec expension de γ_3 pour iteration tres reussie


Geraldine.BasicTrustRegionWithCoeff{Float64}(0.01, 0.8, 0.5, 0.9, 1.5)

In [60]:
stopTests = Geraldine.SingleStopTest[]

Geraldine.SingleStopTest[]

In [61]:
stopIter = Geraldine.MahalanobisTest{Float64}(Dim, 0.05)

Geraldine.MahalanobisTest{Float64}(10, 0.05, 18.307038053275146)

In [62]:
append!(stopTests, [stopIter])

1-element Array{Geraldine.SingleStopTest,1}:
 Geraldine.MahalanobisTest{Float64}(10, 0.05, 18.307038053275146)

In [63]:
sp = Geraldine.StoppingTests(stopTests)

Geraldine.StoppingTests(Geraldine.SingleStopTest[Geraldine.MahalanobisTest{Float64}(10, 0.05, 18.307038053275146)])

In [64]:
TYPE = Float64


# --- Hessian Approx
Hessian = Geraldine.UncomputedHessian{TYPE}
# --- Smoothing
smoothing = Geraldine.NaiveSmoothing()
# --- Sampling
commonVar = true
sam = RandomSamplingDynamicTrueVariance(NMin = NMin, NMax = Nobs(mo), N0=N0, increment=increment,
                                                    commonVar = commonVar,
                                                    smoothing=smoothing,
                                                    subSampling=subSampling)
# --- definition BTR
mybtr = BTRStruct(sp ; sam = typeof(sam), Hessian = Hessian);

# --- Adding to tests
accumulator = Accum_2nd_tv(xstar)
test = SecondOrderTest(mybtr, sam, btrCoeffs, accumulator; verbose = verbose);
addTest(test, alltests)
println(sam)


RandomSamplingDynamicTrueVariance(100, 50, 50, 100, 100000, #undef, true, Geraldine.NaiveSmoothing(0.75, 2.0), ConstantCoeffSubSampling(10000, 0.25), 2, 0.05, 1.6448536269514717)


In [65]:
TYPE = Float64


# --- Hessian Approx
Hessian = Geraldine.BHHHScores{TYPE}
# --- Smoothing
smoothing = Geraldine.NaiveSmoothing()
# --- Sampling
commonVar = true
sam = RandomSamplingDynamicTrueVariance(NMin = NMin, NMax = Nobs(mo), N0=N0, increment=increment,
                                                    commonVar = commonVar,
                                                    smoothing=smoothing,
                                                    subSampling=subSampling)
# --- definition BTR
mybtr = BTRStruct(sp ; sam = typeof(sam), Hessian = Hessian);

# --- Adding to tests
accumulator = Accum_2nd_tv(xstar)
test = SecondOrderTest(mybtr, sam, btrCoeffs, accumulator; verbose = verbose);
addTest(test, alltests)
println(sam)


RandomSamplingDynamicTrueVariance(100, 50, 50, 100, 100000, #undef, true, Geraldine.NaiveSmoothing(0.75, 2.0), ConstantCoeffSubSampling(10000, 0.25), 2, 0.05, 1.6448536269514717)


In [66]:

  # F Normal
  function Sofia.F(x::Vector{T}, mo::Amlet.LogitModel{Amlet.NotUpdatable, D}; sample = 1:nInd, update::Bool = false) where {T, D}
      return Amlet.F_normal(x, mo; sample = sample)
  end

  lancementAllTests(alltests.testsNormal, alltests)


  # F variance
  function Sofia.F(x::Vector{T}, mo::Amlet.LogitModel{Amlet.NotUpdatable, D}; sample = 1:nInd, update::Bool = false) where {T, D}
      return Amlet.F_variance(x, mo; sample = sample)
  end

  lancementAllTests(alltests.testsVariance, alltests)


 ------- Debut algo btr True Variance ------ : 
 --- sampling : RandomSamplingDynamicTrueVariance
 --- smoothing : Geraldine.NaiveSmoothing(0.75, 2.0)
-------------------------
initializeState! for TRUE Variance
MahalanobisTest
 ------- Debut algo btr True Variance ------ : 
 --- sampling : RandomSamplingDynamicTrueVariance
 --- smoothing : Geraldine.NaiveSmoothing(0.75, 2.0)
-------------------------
initializeState! for TRUE Variance
MahalanobisTest


In [67]:
plotlyjs()

lines = [:solid, :dot, :dash, :dashdot]
extension = "Logit.html"

"Logit.html"

In [71]:

####################################################################################
p = plot(xlabel="iterations", ylabel="Distance to solution", legend=:outertopright)

indicesVariance = 1:2
indicesNormal = []

for i in indicesVariance
    plot!(p, alltests.all_dataVariance[i].IterAccumulator, alltests.all_dataVariance[i].DistTo,
                label = alltests.all_legendVariance[i], line=lines[i%4 + 1])
end


for i in indicesNormal
    plot!(p, alltests.all_dataNormal[i].IterAccumulator, alltests.all_dataNormal[i].DistTo,
                label = alltests.all_legendNormal[i], line=lines[i%4 + 1])
end

fileName = "comparaison_all_methods_distTo"
savefig(p, string(fileName, extension))

p

<!DOCTYPE html>
 
 
 Plots.jl

In [72]:

####################################################################################
p = plot(xlabel="iterations", ylabel="Sample size", legend=:outertopright)

indicesVariance = 1:2
indicesNormal = []

for i in indicesVariance
    plot!(p, alltests.all_dataVariance[i].IterAccumulator, alltests.all_dataVariance[i].SamplingSizeAccumulator,
                label = alltests.all_legendVariance[i], line=lines[i%4 + 1])
end


for i in indicesNormal
    plot!(p, alltests.all_dataNormal[i].IterAccumulator, alltests.all_dataNormal[i].SamplingSizeAccumulator,
                label = alltests.all_legendNormal[i], line=lines[i%4 + 1])
end

fileName = "comparaison_all_methods_distTo"
savefig(p, string(fileName, extension))

p

<!DOCTYPE html>
 
 
 Plots.jl

In [73]:

####################################################################################
p = plot(xlabel="iterations", ylabel="Iter CG", legend=:outertopright)

indicesVariance = 1:2
indicesNormal = []

for i in indicesVariance
    scatter!(p, alltests.all_dataVariance[i].IterAccumulator[2:end], alltests.all_dataVariance[i].Field_iterCG,
                label = alltests.all_legendVariance[i], line=lines[i%4 + 1])
end


for i in indicesNormal
    plot!(p, alltests.all_dataNormal[i].IterAccumulator, alltests.all_dataNormal[i].SamplingSizeAccumulator,
                label = alltests.all_legendNormal[i], line=lines[i%4 + 1])
end

fileName = "comparaison_all_methods_distTo"
savefig(p, string(fileName, extension))

p

<!DOCTYPE html>
 
 
 Plots.jl